<a href="https://colab.research.google.com/github/comet-toolkit/comet_training/blob/main/punpy_digital_effects_table_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Example using digital effects tables**

In this notebook, we will show how digital effects tables, created with obsarray (see [this notebook](https://colab.research.google.com/github/comet-toolkit/comet_training/blob/master/defining_digital_effects_table.ipynb)), can be propagated through a measurement function using punpy. Here we use an example where we calculate the uncertainties in a volume of gas, using the ideal gas law and a digital effects table quantifying the uncertainties and error-correlation of the gas temperature, pressure and amount of substance (number of moles).

We first install and import the obsarray and punpy packages (and xarray, numpy and matplotlib):

In [ ]:
!pip install punpy
!pip install obsarray

In [ ]:
import obsarray
import punpy

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

Next, we clone the CoMet training repository, so we have access to the example digital effects table file, and then read it:

In [ ]:
!git clone https://github.com/comet-toolkit/comet_training.git

In [ ]:
ds = xr.open_dataset("comet_training/digital_effects_table_gaslaw_example.nc")  # read digital effects table

This digital effects table has multiple uncertainty components for each of the included variables. The input quantities and their uncertainties have thus been fully defined within this obsarray object. The only thing that remains to be done before we can propagate the uncertainties is to define the measurement function. In order to be able to use the digital effects tables, this measurement function needs to be defined by subclassing the MeasurementFunction imported from punpy, and then xx the function called meas_function. Here we make a measurement function that implements the ideal gas law:

In [ ]:
from punpy import MeasurementFunction

# Define your measurement function inside a subclass of MeasurementFunction
class IdealGasLaw(MeasurementFunction):
    def meas_function(self, pres, temp, n):
        return (n *temp * 8.134)/pres

Once we have defined the IdealGasLaw, we can create an object of this class. The initialiser of the MeasurementFunction class needs a number of arguments. The first argument is a punpy propagation object, which defines whether the Monte Carlo (MC) or Law of Propagation of Uncertainty (LPU) method is used. For this propagation object, there are a number of optional arguments relevant to the MC or LPU methods, or which provide additional functionality (see later). Here, we set the number of MC steps to 10000. The second argument is a list with the names of the variables in the digital effects table dataset (ds) that are used in the meas_function (and in the order that they are used in the meas_function). The third argument is the name of the measurand. There is also an optional argument to specify the units of the measurand.

In [ ]:
from punpy import MCPropagation

prop = MCPropagation(10000)

gl = IdealGasLaw(prop, ["pressure", "temperature", "n_moles"], yvariable="volume", yunit="m^3")

In [ ]:
ds_y = gl.propagate_ds(ds)
print(ds_y)